In [5]:
import pandas as pd
import random
import hashlib
from data_functions import fake
from oracle_connection import get_engine

random.seed(42)

engine = get_engine()

In [11]:
SENSOR_QUERY = "SELECT S.ID_SENSOR, S.TP_SENSOR, S.TP_MEDIDA, E.ST_ESTACAO FROM T_FV_SENSOR S INNER JOIN T_FV_ESTACAO_TRATAMENTO E ON S.ID_ESTACAO_TRATAMENTO = E.ID_ESTACAO_TRATAMENTO"

df_iot = pd.read_sql_query(SENSOR_QUERY, engine)
df_iot.head()

,id_sensor,tp_sensor,tp_medida,st_estacao
0,11,PH,PH,A
1,12,TURBIDEZ,NTU,A
2,13,TEMPERATURA,CELSIUS,A
3,14,NIVEL,CM,A
4,15,VAZAO,LPM,A


In [12]:
df_iot_ativo = df_iot[df_iot['st_estacao'] == 'A'].copy()
df_iot_ativo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175 entries, 0 to 249
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_sensor   175 non-null    int64 
 1   tp_sensor   175 non-null    object
 2   tp_medida   175 non-null    object
 3   st_estacao  175 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.8+ KB


In [39]:
def retorna_valor(tp_sensor):
    match tp_sensor:
        case 'PH':
            return round(random.uniform(6.5, 8.5), 4)
        case 'TURBIDEZ':
            return round(random.uniform(0.1, 5.0), 4)
        case 'TEMPERATURA':
            return round(random.uniform(15.0, 30.0), 4)
        case 'NIVEL':
            return round(random.uniform(10.0, 100.0), 4)
        case 'VAZAO':
            return round(random.uniform(5.0, 80.0), 4)

        

In [40]:
df_iot_ativo['dt_registro'] = ''
df_iot_ativo['dt_registro'] = df_iot_ativo['dt_registro'].apply(lambda x: fake.date_time_this_month())
df_iot_ativo['nr_resultado'] = df_iot_ativo['tp_sensor'].apply(lambda sensor: retorna_valor(sensor))
df_iot_ativo

,id_sensor,tp_sensor,tp_medida,st_estacao,dt_registro,nr_resultado,id_registro
0,11,PH,PH,A,2025-05-22 17:29:45,7.3021,1260be8e49164619116c7b8b7d94f7ee7a9296adbca8f5...
1,12,TURBIDEZ,NTU,A,2025-05-28 13:49:18,0.3873,8d3a5ecb7e518185c2ff2d64ea74d081740578f80f9903...
2,13,TEMPERATURA,CELSIUS,A,2025-05-10 23:22:12,20.6846,3a75e778338d1e43d278b6a7f2fe0320d27670f7f96c2e...
3,14,NIVEL,CM,A,2025-05-08 23:07:59,98.6778,6152e813109d09fd06e19f39005f903d323818a4315d73...
4,15,VAZAO,LPM,A,2025-05-10 23:37:57,24.8902,c52cd8d817ca395cb1f569109f328d8e1045f43d66ed91...
...,...,...,...,...,...,...,...
245,256,PH,PH,A,2025-05-23 14:14:13,7.8285,6022406a1345bee4951cf2e9e8f7e1443131d1af13274d...
246,257,TURBIDEZ,NTU,A,2025-05-16 11:13:39,1.7180,2c28a6ab00d6515cc9e7b8b7136ba9ede337250247be5e...
247,258,TEMPERATURA,CELSIUS,A,2025-05-19 00:54:49,19.7087,8380482f305ffb593344db9a4b6da1f508b85ca988958c...
248,259,NIVEL,CM,A,2025-05-03 19:46:48,86.3214,dea25a0b1c968e76770aa8a70f0349a91ba512f0bc4dcf...


In [41]:
def gerar_hash(row):
    texto = f"{row['id_sensor']}-{row['dt_registro']}"
    return hashlib.sha256(texto.encode()).hexdigest()
df_iot_ativo['id_registro'] = df_iot_ativo.apply(lambda row: gerar_hash(row), axis=1)

df_iot_ativo.drop_duplicates(subset='id_registro', inplace=True)
df_iot_ativo

,id_sensor,tp_sensor,tp_medida,st_estacao,dt_registro,nr_resultado,id_registro
0,11,PH,PH,A,2025-05-22 17:29:45,7.3021,bbb00a38f1f3a3e9d1e839ab9b086b44142e2a563e7416...
1,12,TURBIDEZ,NTU,A,2025-05-28 13:49:18,0.3873,929100b80ce049d0412027fd9da4feb2ce5e4b7987e3ad...
2,13,TEMPERATURA,CELSIUS,A,2025-05-10 23:22:12,20.6846,fb35d234998aa1c3f657bbb2b039e93d995cf3a1186127...
3,14,NIVEL,CM,A,2025-05-08 23:07:59,98.6778,476e7fbf27b867b3aa6237360e1a323216576dbf1d0982...
4,15,VAZAO,LPM,A,2025-05-10 23:37:57,24.8902,2ff6e7b00b77c96442d5f731d82b9c8c52e572bedca9c2...
...,...,...,...,...,...,...,...
245,256,PH,PH,A,2025-05-23 14:14:13,7.8285,dd58611dee8002788ad0b87adbed1b251629de48c2882b...
246,257,TURBIDEZ,NTU,A,2025-05-16 11:13:39,1.7180,be2d157832acb3dba7189cb4ef0f756f47745a902abcfa...
247,258,TEMPERATURA,CELSIUS,A,2025-05-19 00:54:49,19.7087,e8288d6577cd8a868d926693d315ab56fecde4c3b8116e...
248,259,NIVEL,CM,A,2025-05-03 19:46:48,86.3214,8f83374f5aba7b9eb3a02e231b6a02af57ee4013b3d320...


In [45]:
inserts = []

for _, row in df_iot_ativo.iterrows():
    linha_sql = f"""
    INSERT INTO T_FV_REGISTRO_MEDIDA (
        ID_REGISTRO, DT_REGISTRO, NR_RESULTADO, ID_SENSOR
    ) VALUES (
        '{row['id_registro']}', 
        TO_DATE('{row['dt_registro'].strftime('%Y-%m-%d %H:%M:%S')}', 'YYYY-MM-DD HH24:MI:SS'),
        {row['nr_resultado']}, {row['id_sensor']}
    );
    """.strip()
    inserts.append(linha_sql)

# Exemplo de como salvar em um arquivo:
with open("inserts_registro.sql", "w", encoding="utf-8") as f:
    f.write("BEGIN" + "\n")
    f.write("\n".join(inserts))
    f.write("\n")
    f.write("    " + "COMMIT;" + "\n")
    f.write("EXCEPTION " + "\n")
    f.write("    " + "WHEN OTHERS THEN" + "\n")
    f.write("        " + "ROLLBACK;" + "\n")
    f.write("END;" + "\n")
    f.write("/" + "\n")

In [48]:
df_registros = pd.read_sql_query("SELECT * FROM T_FV_REGISTRO_MEDIDA", engine)
df_registros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id_registro   175 non-null    object        
 1   dt_registro   175 non-null    datetime64[ns]
 2   nr_resultado  175 non-null    float64       
 3   id_sensor     175 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 5.6+ KB
